In [ ]:
from videolabeler import utils as vl
import numpy as np
import cv2
from tqdm.notebook import tqdm
import pandas as pd
import os

In [ ]:
animal_ids = ['OM14', 'OM16']
root_dir = '/Users/mtugsbayar/Downloads/vid-label/tiffs/'

batch_size = 500
label_dict= {'i':'INTERP','s':'still','r':'rearing','w':'walking', 'q':'left turn', 'e':'right turn',
             'a':'left turn [still]', 'd': 'right turn [still]', 'g':'grooming','m':'eating', 't':'explore', 'l':'leap'}

#This is optional. MultiLabeler creates a master_labels.csv file in your root directory if you don't 
labels_file = '/Users/mtugsbayar/Downloads/vid-label/tiffs/master_labels.csv'

In [ ]:
vl.multiLabelerBatchLabel(root_dir,animal_ids,labels_file=labels_file,batch_size=500, n_overlap_frames=25,min_gap=200)

In [ ]:
labels = pd.read_csv(labels_file)
labels.head()